In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Purpose

I am going to go through the task submissions for [Contest #9: UFC Fight Night Smith Vs Rakic](https://www.kaggle.com/mdabbert/ultimate-ufc-dataset/tasks?taskId=1841)  and see what set of predictions was the most profitable (if any!)

# Submissions

There are 5 submissions.  This is a new record!  Thanks so much dan and BossaNostra for submitting.  Anyone else who is interested in the future just do it!!  The results don't matter, it's just for fun.  Additionally, it is a really good way to learn some machine learning techniques for future event predition.

On to the submissions....

1. The dummy file (gives each fighter a .5 probability of winning. Therefore will bet every underdog with greater than +100 odds.)
2. mdabbert's (my!) submission. A GaussianNB Classifier using the following features: ['R_Reach_cms', 'total_round_dif', 'R_Height_cms', 'R_avg_SIG_STR_pct', 'B_age', 'R_longest_win_streak', 'lose_streak_dif', 'ko_dif', 'R_win_by_Decision_Majority', 'longest_win_streak_dif', 'avg_sub_att_dif', 'R_Weight_lbs', 'sig_str_dif', 'B_Height_cms', 'B_avg_SUB_ATT', 'R_win_by_TKO_Doctor_Stoppage', 'B_draw', 'avg_td_dif', 'R_win_by_Decision_Split', 'age_dif', 'B_current_win_streak', 'R_odds']
3. BossaNostra's submission #1
4. BossaNostra's submission #2
5. dan's submission

# Open All Submissions and add to a list

In [ ]:
#This will help us keep track of the submissions
sub_name_list = ['dummy', 'mdabbert', 'BossaNostra 1', 'BossaNostra 2', 'dan']
score_list = [] #We can keep the scores here

#Put the submissions in dataframe form and add to a list.
sub_list = []
temp_df = pd.read_csv("/kaggle/input/ufc-fight-night-smith-vs-rakic-dummy-task-sub/task-dummy.csv")
sub_list.append(temp_df)

temp_df = pd.read_csv("/kaggle/input/ufc-fight-night-82920-contest-submission/ufc-8-29-20-sub.csv")
sub_list.append(temp_df)

temp_df = pd.read_csv("/kaggle/input/smith-vs-rakic-contest-submission-1/Smith_vs_Rakic_Submission_1.csv")
sub_list.append(temp_df)

temp_df = pd.read_csv("/kaggle/input/smith-vs-rakic-contest-submission-2/Smith_vs_Rakic_Submission_2.csv")
sub_list.append(temp_df)

temp_df = pd.read_csv("/kaggle/input/my-entry-for-smith-vs-rakic/dans_sample_submission_8.29.2020.csv")
sub_list.append(temp_df)


In [ ]:
results_df = pd.read_csv("/kaggle/input/ultimate-ufc-dataset/most-recent-event.csv")

#We only need the fighter names, odds, and winner

results_df = results_df[['R_fighter', 'B_fighter', 'R_ev', 'B_ev', 'Winner']]
display(results_df)

For the third week in a row `Blue` performed better than expected, but only slightly this time. (Currently `Red` wins 58% of fights)

# Iterate the submissions and see how everyone did!

In [ ]:
#Returns a specific bet EV based on winning_ev and probability.
def get_bet_ev(ev, prob):
    
    return(ev*prob - (1-prob)*100)

In [ ]:
#Used to determine the bet of each fight.  We will use probabilities and the ev to 
#determine profitable bets
def get_bet(R_prob, B_prob, R_ev, B_ev):
    red_ev = get_bet_ev(R_ev, R_prob)
    blue_ev = get_bet_ev(B_ev, B_prob)
    if red_ev > 0:
        return('Red')
    if blue_ev > 0:
        return('Blue')
    
    return 'None'

In [ ]:
def get_profit(winner, bet, R_ev, B_ev):
    if bet == 'None':
        return 0
    if (bet == 'Blue' and winner == 'Blue'):
        return B_ev
    if (bet == 'Red' and winner == 'Red'):
        return R_ev
    else:
        return (-100)

In [ ]:
#Let's make a helper function to make this easier

def get_score(sub, results):
#    display(sub)
#    display(results)
    #Let's merge the two dataframes
    merge_df = pd.merge(sub, results)
    #display(merge_df)
    #We can get the proper bet by using a lambda function
    merge_df['Bet'] = merge_df.apply(lambda x: get_bet(x['R_prob'],x['B_prob'],x['R_ev'],x['B_ev']), axis=1)
    merge_df['Profit'] = merge_df.apply(lambda x: get_profit(x['Winner'], x['Bet'], x['R_ev'], x['B_ev']), axis=1)
    display(merge_df)
    return(sum(merge_df['Profit']))

# Submission #1: Dummy Submission

These are the results of the dummy submission.

In [ ]:
z = 0
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

Since only 2 underdogs won on Saturday, and only one had a return large enough for dummy to bet on him dummy got hammered.  Losing 690 units on 900 units bet.  A very bad night.

# Submission #2: mdabbert's Submission

In [ ]:
z = 1
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

mdabbert saw a mild profit of 57.06 units.  He won 7 out of 10 bets.  This shows how hard it can be to turn a good profit when only a couple of underdogs come in.

# Submission #3: BossaNostra's 1st Submission

In [ ]:
z = 2
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

BossaNostra won his first two bets and then lost 4 in a row for a return of -268.02 units.

# Submission #4: BossaNostra's 2nd Submission

In [ ]:
z = 3
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

BossaNostra's 2nd submission won 3 of 7 bets for a return of -228.40 units.

# Submission #5: dan's Submission

In [ ]:
z = 4
score_list.append(get_score(sub_list[z], results_df))
print(f"{sub_name_list[z]}'s bets saw a total profit of {score_list[z]}")

dan's model relied on underdogs coming in.  Something that did not happen on Saturday night.  He won 1 of 9 bets for a return of -708.25 units.

# Final Results

* `dummy`: -690.00 units
* `mdabbert`: +57.06 units
* `BossaNostra 1`: -268.02 units
* `BossaNostra 2`: -228.40 units
* `dan`: -708.25 units

`mdabbert` is the winner